# 2023-08-02__Julia_speedtest_AdEx_Nto1

(We're reusing code from `2023-02-24__multisim-winline.jl` and `2023-03-14__[setup]_Nto1_sim_AdEx.jl`).

In [9]:
cd(joinpath(homedir(), "phd", "pkg" , "SpikeWorks"))
run(`git switch metdeklak`);

Your branch is up to date with 'origin/metdeklak'.


Already on 'metdeklak'


In [10]:
cd(joinpath(homedir(), "phd"))
Pkg.activate(".")

  Activating project at `C:\Users\tfiers\phd`


In [11]:
]st

Status `C:\Users\tfiers\phd\Project.toml`
⌃ [1520ce14] AbstractTrees v0.4.3
⌃ [336ed68f] CSV v0.10.9
  [8be319e6] Chain v0.5.0
⌃ [da1fd8a2] CodeTracking v1.2.0
⌃ [b0b7db55] ComponentArrays v0.13.6
⌃ [8f4d0f93] Conda v1.7.0
  [03fa6f02] ConnTestEval v0.1.0 `pkg\ConnTestEval`
  [4ec9db02] ConnectionTests v0.1.0 `pkg\ConnectionTests.jl`
⌃ [f68482b8] Cthulhu v2.7.6
⌃ [a93c6f00] DataFrames v1.4.4
⌃ [1313f7d8] DataFramesMeta v0.12.0
⌃ [864edb3b] DataStructures v0.18.13
  [31a5f54b] Debugger v0.7.8
  [019de50b] DistributedLoopMonitor v0.1.0 `pkg\DistributedLoopMonitor`
⌃ [31c24e10] Distributions v0.25.80
  [48062228] FilePathsBase v0.9.20
⌃ [f6369f11] ForwardDiff v0.10.34
  [4356e881] GlobalMacros v0.1.0 `pkg\GlobalMacros`
  [4c0ca9eb] Gtk v1.3.0
⌃ [7073ff75] IJulia v1.24.0
⌃ [033835bb] JLD2 v0.4.29
  [984bce1d] LambertW v0.4.6
⌃ [23fbe1c1] Latexify v0.15.18
  [2fda8390] LsqFit v0.13.0 `https://github.com/JuliaNLSolvers/LsqFit.jl#e9b9e8732`
  [1914dd2f] MacroTools v0.5.10
  [40c6d27e] MemDisk

In [12]:
using WithFeedback

In [15]:
@withfb using SpikeWorks
@withfb using SpikeWorks.Units

using SpikeWorks … 

[ Info: Precompiling SpikeWorks [fe4ab31d-2284-4e18-9761-4109e720cf88]


✔ (5.3 s)
using SpikeWorks.Units … ✔ (1.5 s)


In [17]:
@typed begin
    # AdEx LIF neuron params (cortical RS)
    C  = 104  * pF
    gₗ = 4.3  * nS
    Eₗ = -65  * mV
    Vₜ = -52  * mV
    Δₜ = 0.8  * mV
    Vₛ =   0  * mV
    Vᵣ = -53  * mV
    a  = -0.8 * nS
    b  =  65  * pA
    τw =  88  * ms
    # Conductance-based synapses
    Eₑ =   0 * mV
    Eᵢ = -80 * mV
    τ  =   7 * ms
end;

### Simulated variables and their initial values

In [38]:
const x₀ = (
    # AdEx variables
    v   = Eₗ,      # Membrane potential
    w   = 0 * pA,  # Adaptation current
    # Synaptic conductances g
    gₑ  = 0 * nS,  # = Sum over all exc. synapses
    gᵢ  = 0 * nS,  # = Sum over all inh. synapses
)

(v = -0.065, w = 0.0, gₑ = 0.0, gᵢ = 0.0)

### Differential equations:
calculate time derivatives of simulated vars  
(and store them "in-place", in `Dₜ`).

In [39]:
function f!(Dₜ, vars)
    v, w, gₑ, gᵢ = vars

    # Conductance-based synaptic current
    Iₛ = gₑ*(v-Eₑ) + gᵢ*(v-Eᵢ)

    # AdEx 2D system
    Dₜ.v = (-gₗ*(v-Eₗ) + gₗ*Δₜ*exp((v-Vₜ)/Δₜ) - Iₛ - w) / C
    Dₜ.w = (a*(v-Eₗ) - w) / τw

    # Synaptic conductance decay
    Dₜ.gₑ = -gₑ / τ
    Dₜ.gᵢ = -gᵢ / τ
end;

### Spike discontinuity

In [40]:
has_spiked(vars) = (vars.v > Vₛ)

function on_self_spike!(vars)
    vars.v = Vᵣ
    vars.w += b
end;

### Conductance-based AdEx neuron

In [41]:
const coba_adex_neuron = NeuronModel(x₀, f!; has_spiked, on_self_spike!);

### More parameters, and input spikers

In [43]:
using SpikeWorks: LogNormal

# Firing rates λ for the Poisson inputs
const fr_distr = LogNormal(median = 4Hz, g = 2)

@typed begin
    Δt = 0.1ms
    EIratio = 4//1
end;

In [44]:
N = 6500
δ_nS = 0.02
(; Nₑ, Nᵢ) = EIMix(N, EIratio)

EIMix of 6500 neurons
- 5200 excitatory (80%)
- 1300 inhibitory (20%)
- 4:1 EI-ratio


In [45]:
duration = 10*minutes

600.0

In [46]:
firing_rates = rand(fr_distr, N)
input_IDs = 1:N
inputs = [
    Nto1Input(ID, poisson_SpikeTrain(λ, duration))
    for (ID, λ) in zip(input_IDs, firing_rates)
];

In [47]:
neuron_type(ID) = (ID ≤ Nₑ) ? :exc : :inh
Δgₑ = δ_nS * nS
Δgᵢ = δ_nS * nS * EIratio

8.000000000000001e-11

In [48]:
using Random
Random.seed!(1);

In [49]:
on_spike_arrival!(vars, spike) =
    if neuron_type(source(spike)) == :exc
        vars.gₑ += Δgₑ
    else
        vars.gᵢ += Δgᵢ
    end;

In [55]:
using SpikeWorks: newsim, run!

sim = newsim(coba_adex_neuron, inputs, on_spike_arrival!, Δt)

SpikeWorks.Simulation{Nto1System{NeuronModel{NamedTuple{(:v, :w, :gₑ, :gᵢ), NTuple{4, Float64}}, typeof(f!), typeof(has_spiked), typeof(on_self_spike!)}, typeof(on_spike_arrival!)}, CVec{(:v, :w, :gₑ, :gᵢ)}}
Summary: not started
Properties: 
       system: Nto1System, x₀: (v = -0.065, w = 0.0, gₑ = 0.0, gᵢ = 0.0), input feed: 0/19847270 spikes processed
           Δt: 0.0001
     duration: 600.0
  stepcounter: 0/6000000
        state: t = 0 seconds, neuron = vars: (v: -0.065, w: 0.0, gₑ: 0.0, gᵢ: 0.0), Dₜvars: (v: 0.0, w: 0.0, gₑ: 0.0, gᵢ: 0.0)
          rec: v: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], spiketimes: Float64[]


In [56]:
@time run!(sim)

  9.063747 seconds (95.55 M allocations: 2.854 GiB, 14.09% gc time)


SpikeWorks.Simulation{Nto1System{NeuronModel{NamedTuple{(:v, :w, :gₑ, :gᵢ), NTuple{4, Float64}}, typeof(f!), typeof(has_spiked), typeof(on_self_spike!)}, typeof(on_spike_arrival!)}, CVec{(:v, :w, :gₑ, :gᵢ)}}
Summary: completed. 10 spikes/s
Properties: 
       system: Nto1System, x₀: (v = -0.065, w = 0.0, gₑ = 0.0, gᵢ = 0.0), input feed: all 19847270 spikes processed
           Δt: 0.0001
     duration: 600.0
  stepcounter: 6000000 (complete)
        state: t = 600 seconds, neuron = vars: (v: -0.054943, w: 6.43374e-11, gₑ: 3.76597e-9, gᵢ: 3.13911e-9), Dₜvars: (v: 0.21907, w: -8.2327e-10, gₑ: -5.45792e-7, gᵢ: -4.54944e-7)
          rec: v: [-0.0649975, -0.0649912, -0.0649824, -0.0649724, -0.0649625, -0.0649466, -0.0649234, -0.0649006, -0.0648757, -0.0648524  …  -0.0551448, -0.0551268, -0.0551054, -0.0550817, -0.0550596, -0.05504, -0.055016, -0.0549895, -0.0549649, -0.054943], spiketimes: [0.0272, 0.0941, 0.1877, 0.2649, 0.3973, 0.5414, 0.6157, 0.7515, 0.8477, 0.9215  …  599.118, 599.236,

In [58]:
_spiketimes(input::Nto1Input) = input.train.spiketimes

simdata = (;
    spiketrains   = _spiketimes.(inputs),
    voltsig       = sim.rec.v,
    spikerate     = SpikeWorks.spikerate(sim),
    input_types   = neuron_type.(input_IDs),
    sim_duration  = duration,
    firing_rates, input_IDs, N, δ_nS
);